# Initializing Colab

Installing pytorch_xla

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

Downloading data

In [ ]:
!wget https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/prediction/v1.1/{sample.tar,aerial_map.tar,semantic_map.tar}
!wget https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/prediction/v1.1/{train.tar,validate.tar} 
!mkdir -p lyft/scenes/ lyft/aerial_map/ lyft/semantic_map/
!tar -xf sample.tar sample.zarr && mv sample.zarr lyft/scenes/ && rm sample.tar
!tar -xf aerial_map.tar -C lyft/ && rm aerial_map.tar
!tar -xf semantic_map.tar meta.json && mv meta.json lyft/
!tar -xf semantic_map.tar semantic_map.pb && mv semantic_map.pb lyft/semantic_map/ && rm semantic_map.tar

Cloning repository and installing dependencies

In [ ]:
!git clone https://github.com/VahidZee/ReasonAwareRasterizedTrajectoryPrediction.git
cd ReasonAwareRasterizedTrajectoryPrediction
pip install -r requirements.txt

# Importing dependencies

In [1]:
import pytorch_lightning as pl
from l5kit.configs import load_config_data
from raster import BaseResnet, BaseTrainerModule, LyftDataModule

## Initializing various parts

In [2]:
config = load_config_data('./config.yaml')
datamodule = LyftDataModule('/Users/sana/Desktop/dataset_lyft/', config)
training_procedure = BaseTrainerModule(config)
training_procedure.datamodule = datamodule

In [3]:
checkpointing = pl.callbacks.ModelCheckpoint(verbose=True, monitor='loss/val')

In [4]:
trainer = pl.Trainer(fast_dev_run = True,max_epochs = 5,limit_train_batches=2, limit_val_batches=2, checkpoint_callback=checkpointing, **config['train_params'].get('trainer', dict()), )

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [5]:
!kill 12111

/bin/sh: line 0: kill: (12111) - No such process


In [6]:
trainer.fit(training_procedure)

meta.json is not present in local data folder /Users/sana/Desktop/dataset_lyft
!!dataset metafile not found!! the hard-coded matrix will be loaded.
This will be deprecated in future releases


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name      | Type       | Params
-----------------------------------------
0 | model     | BaseResnet | 23 M  
1 | criterion | MSELoss    | 0     
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarn

here
0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/captum/attr/_utils/gradient.py:33: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  "required_grads has been set automatically." % index


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
here
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: 
                    When using EvalResult(early_stop_on=X) or TrainResult(early_stop_on=X) the
                    'monitor' key of ModelCheckpoint has no effect.
                    Remove ModelCheckpoint(monitor='loss/val) to fix')
                
  warnings.warn(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: RuntimeWarning: The metric you returned None must be a `torch.Tensor` instance, checkpoint not saved HINT: what is the value of loss/val in validation_epoch_end()?
  warnings.warn(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: RuntimeWarning: Can save best model only with loss/val available, skipping.
  warnings.warn(*args, **kwargs)
Saving latest checkpo

1